# 🧠 GoogLeNet(InceptionV3) & ResNet50 실습 (Keras Applications)

이 노트북에서는 Keras Applications을 활용하여 GoogLeNet(InceptionV3)과 ResNet50을 실습합니다.

## 주요 내용
1. 사전학습(Pretrained) 모델 불러오기
2. 모델 튜닝 (Fine-tuning)
3. 전이학습 (Transfer Learning)
4. 파라미터 수 비교

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3, ResNet50
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers, models, optimizers
import numpy as np
print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.12.0


## [1] GoogLeNet (InceptionV3) 모델 불러오기

In [2]:
inception = InceptionV3(weights='imagenet', include_top=True)
inception.summary()

96112376/96112376 [==============================] - 10s 0us/step
Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)          

### 예시 이미지로 예측

In [3]:
img_path = tf.keras.utils.get_file('elephant.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/elephant.jpg')
img = image.load_img(img_path, target_size=(299, 299))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = inception.predict(x)
print('예측 결과:', decode_predictions(preds, top=3)[0])

2025-10-14 17:40:32.363716: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 1s 589ms/step
예측 결과: [('n02504458', 'African_elephant', 0.7396527), ('n01871265', 'tusker', 0.13062869), ('n02504013', 'Indian_elephant', 0.03260676)]


## [2] ResNet50 모델 불러오기

In [4]:
resnet = ResNet50(weights='imagenet', include_top=True)
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [5]:
img_res = image.load_img(img_path, target_size=(224, 224))
x_res = image.img_to_array(img_res)
x_res = np.expand_dims(x_res, axis=0)
x_res = tf.keras.applications.resnet50.preprocess_input(x_res)
preds_res = resnet.predict(x_res)
print('ResNet50 예측 결과:', tf.keras.applications.resnet50.decode_predictions(preds_res, top=3)[0])

1/1 [==============================] - 0s 450ms/step
ResNet50 예측 결과: [('n02504458', 'African_elephant', 0.6684765), ('n01871265', 'tusker', 0.24739885), ('n02504013', 'Indian_elephant', 0.08383819)]


## [3] 모델 튜닝 (Fine-tuning)

In [6]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(150,150,3))
base_model.trainable = False

model_ft = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(5, activation='softmax')
])

model_ft.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_ft.summary()

87910968/87910968 [==============================] - 10s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 3, 3, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 128)               262272    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 5)                 645       
                                                                 
Total params: 22,065,701
Trainable params: 262,917
Non-t

In [7]:
for layer in base_model.layers[-30:]:
    layer.trainable = True
model_ft.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

## [4] 전이학습 (Transfer Learning)

In [8]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

feature_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32,32,3))
feature_model.trainable = False

transfer_model = models.Sequential([
    feature_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

transfer_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
transfer_model.summary()

94765736/94765736 [==============================] - 9s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 1, 1, 2048)        23587712  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 256)               524544    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                2570      
                                                                 
Total params: 24,114,826
Trainable params: 527,114
Non-

In [9]:
history = transfer_model.fit(x_train, y_train, epochs=3, batch_size=128, validation_split=0.1, verbose=2)
test_loss, test_acc = transfer_model.evaluate(x_test, y_test, verbose=0)
print(f'전이학습 테스트 정확도: {test_acc:.4f}')

Epoch 1/3
352/352 - 40s - loss: 2.2724 - accuracy: 0.1384 - val_loss: 2.1492 - val_accuracy: 0.2296 - 40s/epoch - 113ms/step
Epoch 2/3
352/352 - 42s - loss: 2.1642 - accuracy: 0.1732 - val_loss: 2.0854 - val_accuracy: 0.2298 - 42s/epoch - 119ms/step
Epoch 3/3
352/352 - 46s - loss: 2.1374 - accuracy: 0.1842 - val_loss: 2.0398 - val_accuracy: 0.2676 - 46s/epoch - 129ms/step
전이학습 테스트 정확도: 0.2663


In [10]:
models_info = {
    'InceptionV3': inception.count_params(),
    'ResNet50': resnet.count_params(),
    'Fine-tuned Inception': model_ft.count_params(),
    'Transfer ResNet50': transfer_model.count_params()
}
for name, params in models_info.items():
    print(f'{name}: {params:,} parameters')

InceptionV3: 23,851,784 parameters
ResNet50: 25,636,712 parameters
Fine-tuned Inception: 22,065,701 parameters
Transfer ResNet50: 24,114,826 parameters
